# LASSO Regresssion (Coordinate Descent)

## Import Module & Load Data

In [1]:
import pandas as pd
import numpy as np
from lasso_func import normalize_features
from lasso_func import predict_output
from lasso_func import lasso_cyclical_coordinate_descent

dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)
testing = pd.read_csv('kc_house_test_data.csv', dtype=dtype_dict)
training = pd.read_csv('kc_house_train_data.csv', dtype=dtype_dict)
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to int, before using it below
sales['floors'] = sales['floors'].astype(int)
sales['constant'] = 1
testing['constant'] = 1
training['constant'] = 1

## Import useful functions from previous notebook

In [2]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

## Normalize Features

In [3]:
def normalize_features(features):
    norms = np.linalg.norm(features, axis = 0)
    X_normalized = features / norms
    return(X_normalized, norms)

## Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function:

$SUM[ (prediction - output)^2 ] + \lambda*( |w[1]| + ... + |w[k]|).$

(By convention, we do not include w[0] in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use coordinate descent: at each iteration, we will fix all weights but weight i and find the value of weight i that minimizes the objective. That is, we look for

$argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + \lambda*( |w[1]| + ... + |w[k]|) ]$

where all weights other than w[i] are held to be constant. We will optimize one w[i] at a time, circling through the weights multiple times.

1. Pick a coordinate i
2. Compute $w[i]$ that minimizes the cost function $SUM[ (prediction - output)^2 ] + \lambda*( |w[1]| + ... + |w[k]|)$
3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent** with normalized features, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above.

## Effect of L1 penalty

In [4]:
simple_features = ['constant','sqft_living','bedrooms']
simple_feature_matrix = sales[simple_features].as_matrix()
output = sales['price'].as_matrix()

simple_feature_matrix , norms = normalize_features(simple_feature_matrix)
weights = np.array([1.,4.,1.])
prediction = predict_output(simple_feature_matrix,weights)



Compute the values of ro[i] for each feature in this simple model, using the formula given above, using the formula:

$\rho[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]$

Hint: You can get a Numpy vector for feature_i using:
simple_feature_matrix[:,i]

$\rho[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]$

because

$Y_i(w-j) = (prediction - weight[i]*[feature_i])$

In [5]:
rho = [0 for i in range((simple_feature_matrix.shape)[1])]
for j in range((simple_feature_matrix.shape)[1]):
    rho[j] = (simple_feature_matrix[:,j] * (output - prediction + (weights[j] * simple_feature_matrix[:,j]))).sum()
    print("Rho: %.4g" %rho[j])

Rho: 7.94e+07
Rho: 8.794e+07
Rho: 8.097e+07


## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [6]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix,weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = (feature_matrix[:,i] * (output - prediction + (weights[i] * feature_matrix[:,i]))).sum()
    
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2.
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2.
    else:
        new_weight_i = 0.
    
    return new_weight_i

## Cyclical coordinate descent

In [7]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    n_feature = feature_matrix.shape[1]
    weights = np.array(initial_weights)
    change = np.array(initial_weights) * 0.0
    converged = False
    
    while not converged:
        # Evaluate over all features
        for idx in range(n_feature):
            # new weights for feature
            new_weight = lasso_coordinate_descent_step(idx,feature_matrix,output,weights,l1_penalty)
            # compute change in weight for feature
            change[idx] = np.abs(new_weight - weights[idx])
            # assign new weight
            weights[idx] = new_weight
            
        max_change = max(change)
        
        if max_change < tolerance:
            converged = True
            
    return weights

Using the following parameters, learn the weights on the sales dataset:

In [8]:
simple_features = ['constant','sqft_living', 'bedrooms']
simple_feature_matrix = sales[simple_features].as_matrix()
output = sales['price'].as_matrix()
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`:

In [9]:
normalized_simple_feature_matrix, simple_norm = normalize_features(simple_feature_matrix)

Then, run your implementation of LASSO coordinate descent:

In [10]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix,output,initial_weights,l1_penalty,tolerance)
print("Weights from cyclical coordinate descent: \n", weights)

prediction =  predict_output(normalized_simple_feature_matrix, weights)
residuals = output - prediction
RSS = (residuals ** 2).sum()
print('RSS for normalized dataset: ', RSS)

Weights from cyclical coordinate descent: 
 [ 21624997.95951909  63157247.20788956         0.        ]
RSS for normalized dataset:  1.63049247672e+15


## Evaluating LASSO fit with more features

Let us consider the following set of features:

In [11]:
all_features = ['constant',
                'bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

In [12]:
feature_matrix = training[all_features].as_matrix()
output = training['price'].as_matrix()
normalized_feature_matrix, norm = normalize_features(feature_matrix)

First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`. Call resulting weights `weights1e7`, you will need them later.

In [13]:
initial_weights = np.zeros(len(all_features))
l1_penalty = 1e7
tolerance = 1.0

weights1e7 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output, initial_weights, l1_penalty, tolerance)
print("Weights with L1 of 1e7: \n", weights1e7)

feature_weights1e7 = dict(zip(all_features, weights1e7))
for k,v in feature_weights1e7.items():
    if v != 0.0:
        print(k, v)

Weights with L1 of 1e7: 
 [ 23864692.5095384          0.                 0.          30495548.13254719
         0.                 0.           1901633.61475594
   5705765.01673266         0.                 0.                 0.
         0.                 0.                 0.        ]
constant 23864692.5095
sqft_living 30495548.1325
waterfront 1901633.61476
view 5705765.01673


Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`. Call resulting weights `weights1e8`, you will need them later.

In [14]:
l1_penalty=1e8
tolerance = 1.0
weights1e8 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output, initial_weights, l1_penalty, tolerance)
print("Weights with L1 of 1e8: \n", weights1e8)

feature_weights1e8 = dict(zip(all_features, weights1e8))
for k,v in feature_weights1e8.items():
    if v != 0.0:
        print(k, v)

Weights with L1 of 1e8: 
 [ 53621004.68971471         0.                 0.                 0.
         0.                 0.                 0.                 0.
         0.                 0.                 0.                 0.
         0.                 0.        ]
constant 53621004.6897


Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`. Call resulting weights `weights1e4`, you will need them later. (This case will take quite a bit longer to converge than the others above.)

In [15]:
l1_penalty=1e4
tolerance=5e5
weights1e4 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output, initial_weights, l1_penalty, tolerance)
print("Weights with L1 of 1e4: \n", weights1e4)

feature_weights1e4 = dict(zip(all_features, weights1e4))
for k,v in feature_weights1e4.items():
    if v != 0.0:
        print(k, v)

Weights with L1 of 1e4: 
 [ 57481091.13302065 -13652628.5402242   12462713.07126242
  57942788.37331226  -1475769.69427563  -4904547.75546551
   5349050.18636169   5845253.56213634   -416038.96981262
   2682274.59488504    242649.68555067  -1285549.66768123
 -54779474.22768356   2167703.06610233]
constant 57481091.133
bedrooms -13652628.5402
bathrooms 12462713.0713
sqft_living 57942788.3733
sqft_lot -1475769.69428
floors -4904547.75547
waterfront 5349050.18636
view 5845253.56214
condition -416038.969813
grade 2682274.59489
sqft_above 242649.685551
sqft_basement -1285549.66768
yr_built -54779474.2277
yr_renovated 2167703.0661


## Rescaling learned weights

In [16]:
feature_matrix = training[all_features].as_matrix()
output = training['price'].as_matrix()
normalized_feature_matrix, norms = normalize_features(feature_matrix)

normalized_weights1e7 = weights1e7 / norms
normalized_weights1e8 = weights1e8 / norms
normalized_weights1e4 = weights1e4 / norms
print("Noramlized weights (L1=1e7): ", normalized_weights1e7)
print("Noramlized weights (L1=1e8): ", normalized_weights1e8)
print("Noramlized weights (L1=1e4): ", normalized_weights1e4)

Noramlized weights (L1=1e7):  [  2.41550915e+05   0.00000000e+00   0.00000000e+00   1.35176516e+02
   0.00000000e+00   0.00000000e+00   2.10000302e+05   6.99212807e+04
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00]
Noramlized weights (L1=1e8):  [ 542734.9516443       0.              0.              0.              0.
       0.              0.              0.              0.              0.
       0.              0.              0.              0.       ]
Noramlized weights (L1=1e4):  [  5.81805533e+05  -3.93707539e+04   5.59594804e+04   2.56840908e+02
  -3.39635078e-01  -3.14486174e+04   5.90703775e+05   7.16306427e+04
  -1.21113113e+03   3.50209970e+03   1.24137861e+00  -2.45031095e+01
  -2.81306954e+02   5.29418824e+01]


## Evaluating each of the learned models on the test data

In [17]:
test_feature_matrix = testing[all_features].as_matrix()
test_output = testing['price']

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [18]:
# RSS (L1 = 1e7)
prediction =  predict_output(test_feature_matrix, normalized_weights1e7)
residuals = test_output - prediction
RSS = (residuals **2).sum()
print('RSS for model with weights1e7: %.4g' %RSS)

RSS for model with weights1e7: 1.631e+14


In [19]:
# RSS (L1 = 1e8)
prediction =  predict_output(test_feature_matrix, normalized_weights1e8)
residuals = test_output - prediction
RSS = (residuals **2).sum()
print('RSS for model with weights1e8: %.4g' %RSS)

RSS for model with weights1e8: 2.847e+14


In [20]:
# RSS (L1 = 1e4)
prediction =  predict_output(test_feature_matrix, normalized_weights1e4)
residuals = test_output - prediction
RSS = (residuals **2).sum()
print('RSS for model with weights1e4: %.4g' %RSS)

RSS for model with weights1e4: 1.291e+14
